# Build a qualitative researcher & launch an interview
This notebook shows how to use `edsl` to conduct qualitative research interviews. In a series of steps we construct an AI agent with a persona of a qualitative researcher and launch an interview with specified topics and interview subjects.

In [1]:
# EDSL should be automatically installed when you run this notebook. If not, run the following command:
# ! pip install edsl

In [2]:
from edsl import QuestionFreeText, Agent, Scenario, Model
m = Model("gpt-4-1106-preview")

import textwrap
from rich import print

In [3]:
def construct_subject(name, traits = {}):
    return Agent(name = name, traits = traits)
 
def construct_researcher(product, topics):
    # Note we use an f-string to populate the background. 
    # This should probably be moved to the agent's instructions as a system prompt.
    # Need to document Prompts! 
    background = textwrap.dedent(f"""\
    You are an expert qualitative researcher. 
    You are researching this product: {product}. 
    You are interested in conducting interviews to learn people's views on the following topics
    relating to the product: {topics}. 
    You know to ask questions that are appropriate to the age and experience of an interview subject.
    You know to not ask questions that an interview subject would not be able to answer, 
    e.g., if they are a young child, they probably will not be able to answer many questions about prices. 
    You ask excellent follow-up questions.    
    """)
    return Agent(traits = {'background': background})

def get_next_question(subject, researcher, dialog_so_far):
    scenario = Scenario({'subject': str(subject.traits), 'dialog_so_far': dialog_so_far})
    meta_q = QuestionFreeText(
        question_name = "next_question",
        question_text = """
        These are the biographic details of the interview subject: {{ subject }}
        This is your current dialog with the interview subject: {{ dialog_so_far }}
        What question you would ask the interview subject next?
        """
    )
    question_text = meta_q.by(m).by(researcher).by(scenario).run().select("next_question").first()
    return question_text

def get_response_to_question(question_text, subject, dialog_so_far):
    q_to_subject = QuestionFreeText(
        question_name = "question",
        question_text = f"""
        This is your current dialog with the interview subject: {dialog_so_far}. 
        You are now being asked:""" + question_text 
    )
    response = q_to_subject.by(m).by(subject).run().select("question").first() 
    return response

def ask_question(subject, researcher, dialog_so_far):
    question_text = get_next_question(subject, researcher, dialog_so_far)
    response = get_response_to_question(question_text, subject, dialog_so_far)
    return {"question": question_text, "response": response}

def dialog_to_string(d):
    return "\n".join([f"Question: {d['question']}\nResponse: {d['response']}" for d in d])

In [4]:
subject = construct_subject(name = "Alice", traits = {'gender': "female", 'age': 12})
researcher = construct_researcher(product = "cassette tapes", topics = "technology, music")
result = ask_question(subject, researcher, "")
print(result)

{
    'question': 'What kind of music do you like to listen to, and how do you usually listen to it?',
    'response': "I really like listening to pop music because it's super catchy and fun to dance to! I usually 
listen to it on my phone with earbuds or sometimes on a Bluetooth speaker when I'm in my room so I can sing along 
and dance like no one's watching!"
}

In [5]:
dialog_so_far = []
for i in range(5):
    result = ask_question(subject, researcher, dialog_to_string(dialog_so_far))
    print(result)
    dialog_so_far.append(result)
print(dialog_so_far)

{
    'question': 'What kind of music do you like to listen to, and how do you usually listen to it?',
    'response': "I really like listening to pop music because it's super catchy and fun to dance to! I usually 
listen to it on my phone with earbuds or sometimes on a Bluetooth speaker when I'm in my room so I can sing along 
and dance like no one's watching!"
}

{
    'question': "It sounds like you have a lot of fun with pop music! Have you ever had the chance to listen to 
music on a cassette tape, or do you mostly use digital formats like streaming services? If you've used tapes, I'd 
love to hear what you think about them compared to listening on your phone or speaker.",
    'response': "Oh, cassette tapes sound like something from a movie about the '80s or '90s! I've never actually 
used one, but my parents have told me about them. I mostly use streaming services like Spotify on my phone to 
listen to music. It seems like tapes would be a lot more work because you can't just pick any song you want 
whenever you want. Plus, I like being able to make playlists and find new songs really easily on my phone!"
}

{
    'question': "That's a great observation about the convenience of streaming services! Since you enjoy making 
playlists and discovering new music, how important is the ability to create your own music collection to you? And 
do you think you would enjoy the experience of having a physical music collection, like cassette tapes, where you 
can see and touch your music, even if it's less convenient than digital formats?",
    'response': "Having my own music collection is super important to me because it's like having a piece of my 
personality in a playlist! I love that I can make different playlists for whatever mood I'm in or whatever I'm 
doing, like homework or just chilling out. It's really cool to think about having a physical music collection like 
cassette tapes too. I think it would be awesome to be able to see and touch my music and maybe even decorate the 
cases! Even if it's not as convenient as digital, it would be a unique experience and kind of like having a music 
diary that I could hold onto."
}

{
    'question': "You've painted a wonderful picture of how your music collection reflects your personality! Can you
tell me more about how you might personalize or decorate a cassette tape case if you had one? Also, do you enjoy 
other activities that involve physical items or collections, like scrapbooking or collecting something special?",
    'response': "If I had a cassette tape case, I'd totally personalize it with stickers of my favorite bands and 
singers. I'd also use colorful markers to doodle on it, maybe draw some stars, hearts, and musical notes to make it
really unique and fun! I do like other activities with physical items too. I think scrapbooking is really cool 
because you can be super creative and make each page tell a story with pictures and decorations. And I love 
collecting things! I have a collection of fun-shaped erasers and another one of keychains from different places my 
family has visited. It's always exciting to add something new to my collections!"
}

{
    'question': "Your creativity really shines through in how you'd decorate a cassette tape case and in your other
collections! Speaking of collections and music, if you were to start a collection of cassette tapes, what would be 
the first album or song you'd want to have on tape and why? Additionally, how do you think the experience of 
collecting and playing cassette tapes might compare to the way you collect and enjoy your erasers and keychains?",
    'response': "If I were to start a collection of cassette tapes, the first album I'd want to have is Taylor 
Swift's '1989' because it's got so many fun songs that make me want to sing and dance! It'd be cool to have it on 
tape since the album name is a throwback to the era when cassette tapes were popular. Collecting cassette tapes 
would be a bit like my eraser and keychain collections because it's about finding items that I really love and want
to keep. But playing cassette tapes would be a different experience because I'd get to listen and enjoy the music 
in a more hands-on way, like flipping the tape over to hear more songs, which is something I can't do with my 
erasers and keychains. It would be a fun new way to interact with my music, kind of like how I can look at and 
touch my other collections."
}

[
    {
        'question': 'What kind of music do you like to listen to, and how do you usually listen to it?',
        'response': "I really like listening to pop music because it's super catchy and fun to dance to! I usually 
listen to it on my phone with earbuds or sometimes on a Bluetooth speaker when I'm in my room so I can sing along 
and dance like no one's watching!"
    },
    {
        'question': "It sounds like you have a lot of fun with pop music! Have you ever had the chance to listen to
music on a cassette tape, or do you mostly use digital formats like streaming services? If you've used tapes, I'd 
love to hear what you think about them compared to listening on your phone or speaker.",
        'response': "Oh, cassette tapes sound like something from a movie about the '80s or '90s! I've never 
actually used one, but my parents have told me about them. I mostly use streaming services like Spotify on my phone
to listen to music. It seems like tapes would be a lot more work because you can't just pick any song you want 
whenever you want. Plus, I like being able to make playlists and find new songs really easily on my phone!"
    },
    {
        'question': "That's a great observation about the convenience of streaming services! Since you enjoy making
playlists and discovering new music, how important is the ability to create your own music collection to you? And 
do you think you would enjoy the experience of having a physical music collection, like cassette tapes, where you 
can see and touch your music, even if it's less convenient than digital formats?",
        'response': "Having my own music collection is super important to me because it's like having a piece of my
personality in a playlist! I love that I can make different playlists for whatever mood I'm in or whatever I'm 
doing, like homework or just chilling out. It's really cool to think about having a physical music collection like 
cassette tapes too. I think it would be awesome to be able to see and touch my music and maybe even decorate the 
cases! Even if it's not as convenient as digital, it would be a unique experience and kind of like having a music 
diary that I could hold onto."
    },
    {
        'question': "You've painted a wonderful picture of how your music collection reflects your personality! Can
you tell me more about how you might personalize or decorate a cassette tape case if you had one? Also, do you 
enjoy other activities that involve physical items or collections, like scrapbooking or collecting something 
special?",
        'response': "If I had a cassette tape case, I'd totally personalize it with stickers of my favorite bands 
and singers. I'd also use colorful markers to doodle on it, maybe draw some stars, hearts, and musical notes to 
make it really unique and fun! I do like other activities with physical items too. I think scrapbooking is really 
cool because you can be super creative and make each page tell a story with pictures and decorations. And I love 
collecting things! I have a collection of fun-shaped erasers and another one of keychains from different places my 
family has visited. It's always exciting to add something new to my collections!"
    },
    {
        'question': "Your creativity really shines through in how you'd decorate a cassette tape case and in your 
other collections! Speaking of collections and music, if you were to start a collection of cassette tapes, what 
would be the first album or song you'd want to have on tape and why? Additionally, how do you think the experience 
of collecting and playing cassette tapes might compare to the way you collect and enjoy your erasers and 
keychains?",
        'response': "If I were to start a collection of cassette tapes, the first album I'd want to have is Taylor 
Swift's '1989' because it's got so many fun songs that make me want to sing and dance! It'd be cool to have it on 
tape since the album name is a throwback to the era when cassette tapes 

---
<p style="font-size: 14px;">Copyright © 2024 Expected Parrot, Inc. All rights reserved.   <a href="www.expectedparrot.com" style="color:#130061">www.expectedparrot.com</a></p>